In [1]:
from torch.utils.data import DataLoader

from src.concept_bottleneck.dataset import CUB200ImageToClass

batch_size = 16
num_workers = 2

test_data = CUB200ImageToClass(train=False)
test_dataloader = DataLoader(test_data, batch_size=batch_size, num_workers=num_workers)


In [2]:
import torch

from src.concept_bottleneck.inference import (
    load_image_to_attributes_model,
    load_attributes_to_class_model,
    INDEPENDENT_IMAGE_TO_ATTRIBUTES_MODEL_NAME,
    INDEPENDENT_ATTRIBUTES_TO_CLASS_MODEL_NAME,
    SEQUENTIAL_ATTRIBUTES_TO_CLASS_MODEL_NAME,
    JOINT_IMAGE_TO_ATTRIBUTES_MODEL_NAME,
    JOINT_ATTRIBUTES_TO_CLASS_MODEL_NAME,
)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

models: dict[str, tuple[torch.nn.Module, torch.nn.Module]] = {
    "independent": (
        load_image_to_attributes_model(
            INDEPENDENT_IMAGE_TO_ATTRIBUTES_MODEL_NAME, device
        ),
        load_attributes_to_class_model(
            INDEPENDENT_ATTRIBUTES_TO_CLASS_MODEL_NAME, device
        ),
    ),
    "sequential": (
        load_image_to_attributes_model(
            INDEPENDENT_IMAGE_TO_ATTRIBUTES_MODEL_NAME, device
        ),
        load_attributes_to_class_model(
            SEQUENTIAL_ATTRIBUTES_TO_CLASS_MODEL_NAME, device
        ),
    ),
    "joint": (
        load_image_to_attributes_model(JOINT_IMAGE_TO_ATTRIBUTES_MODEL_NAME, device),
        load_attributes_to_class_model(JOINT_ATTRIBUTES_TO_CLASS_MODEL_NAME, device),
    ),
}


Using cuda device


Using cache found in /home/shuangwu/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/shuangwu/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/shuangwu/.cache/torch/hub/pytorch_vision_v0.10.0


In [3]:
import numpy as np


def test(
    image_to_attributes_model: torch.nn.Module,
    attributes_to_class_model: torch.nn.Module,
    dataloader: DataLoader[tuple[torch.Tensor, np.int_]],
    loss_fn: torch.nn.Module,
    device: str,
):
    attributes_to_class_model.eval()
    image_to_attributes_model.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for x, y in dataloader:
            x = image_to_attributes_model(x.to(device))
            y = y.to(device)

            logits = attributes_to_class_model(torch.sigmoid(x))
            test_loss += loss_fn(logits, y).item()
            correct += (logits.argmax(dim=1) == y).sum().item()

    test_loss /= len(dataloader)
    accuracy = correct / len(dataloader.dataset)  # type: ignore

    return test_loss, accuracy


In [4]:
for name, (image_to_attributes_model, attributes_to_class_model) in models.items():
    test_loss, accuracy = test(
        image_to_attributes_model,
        attributes_to_class_model,
        test_dataloader,
        torch.nn.CrossEntropyLoss(),
        device,
    )
    print(f"Test loss for {name} model: {test_loss:.2f}")
    print(f"Test accuracy for {name} model: {accuracy:.2f}")


Test loss for independent model: 2.38
Test accuracy for independent model: 0.45
Test loss for sequential model: 1.53
Test accuracy for sequential model: 0.60
Test loss for joint model: 5.28
Test accuracy for joint model: 0.34
